In [6]:
import pandas as pd
import bs4
import requests
import urllib
from selenium import webdriver

[beautiful soup docs](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [10]:
_URL = "https://www.brilliantearth.com/round-diamonds-search/"
driver = webdriver.Chrome()

driver.get(_URL)

soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
# page = urllib.request.urlopen(url=_URL)

Selenium to the rescue!!!

In [16]:
soup

<html class="no-touch mac" lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide{display:none !important;}ng\:form{display:block;}.ng-animate-block-transitions{transition:0s all!important;-webkit-transition:0s all!important;}.ng-hide-add-active,.ng-hide-remove{display:block!important;}</style>
<!-- load third part script content base on url and tag_type -->
<script async="" crossorigin="anonymous" src="https://connect.facebook.net/en_US/all.js?hash=d4834211d7c6cc20ea9f0f145f39e4a1&amp;ua=modern_es6"></script><script async="" src="https://www.googleadservices.com/pagead/conversion_async.js" type="text/javascript"></script><script async="" src="https://s.yimg.com/wi/ytc.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-752176699"></script><script async="" crossorigin="anonymous" integrity="sha384-W4RqaNUbvBdTRc41QQAWDcd2aX9wGruak2WnlXwy